In [ ]:
from pyspark.sql import SparkSession

# 1 Инициализация Spark
spark = SparkSession \
    .builder \
    .appName("ReadFromMinIOToClickHouse") \
    .config(
        "spark.jars.packages",
        ",".join([
            "org.apache.hadoop:hadoop-aws:3.4.1",
            "com.amazonaws:aws-java-sdk-bundle:1.12.262",
            "com.clickhouse:clickhouse-jdbc:0.9.1"
            # "com.clickhouse:clickhouse-client:0.6.2",
            # "ru.yandex.clickhouse:clickhouse-jdbc:0.3.2"
        ])
    ) \
    .config("spark.hadoop.fs.s3a.access.key", "minioadmin") \
    .config("spark.hadoop.fs.s3a.secret.key", "minioadmin") \
    .config("spark.hadoop.fs.s3a.endpoint", "http://minio:9000") \
    .config("spark.hadoop.fs.s3a.path.style.access", "true") \
    .config("spark.hadoop.fs.s3a.connection.ssl.enabled", "false") \
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
    .getOrCreate()

spark.sparkContext.setLogLevel("WARN")

df = spark.read.parquet("s3a://data/orders_stream/")

df_to_write = df.selectExpr(
"key_id as key_id",
"operation as operation",
"id as id",
"order_id as order_id",
"status as status",
"ts_data as ts_data",
"ts_event_ns as ts_event_ns"
)

clickhouse_url = "jdbc:clickhouse://clickhouse:8123/default"

df_to_write.write \
    .format("jdbc") \
    .option("url", clickhouse_url) \
    .option("driver", "com.clickhouse.jdbc.ClickHouseDriver") \
    .option("dbtable", "orders_stream") \
    .option("user", "default") \
    .option("password", "your_password") \
    .mode("append") \
    .save()